<a href="https://colab.research.google.com/github/Kokoabassplayer/Speech-to-text-with-Whisper-model/blob/main/%E0%B9%80%E0%B8%9B%E0%B8%A5%E0%B8%B5%E0%B9%88%E0%B8%A2%E0%B8%99%E0%B9%80%E0%B8%AA%E0%B8%B5%E0%B8%A2%E0%B8%87%E0%B9%80%E0%B8%9B%E0%B9%87%E0%B8%99%E0%B8%82%E0%B9%89%E0%B8%AD%E0%B8%84%E0%B8%A7%E0%B8%B2%E0%B8%A1%E0%B8%87%E0%B9%88%E0%B8%B2%E0%B8%A2%E0%B9%86_%E0%B8%94%E0%B9%89%E0%B8%A7%E0%B8%A2_Whisper_%E0%B8%84%E0%B8%B9%E0%B9%88%E0%B8%A1%E0%B8%B7%E0%B8%AD%E0%B8%81%E0%B8%B2%E0%B8%A3%E0%B8%96%E0%B8%AD%E0%B8%94%E0%B9%80%E0%B8%AA%E0%B8%B5%E0%B8%A2%E0%B8%87%E0%B9%83%E0%B8%AB%E0%B9%89%E0%B9%80%E0%B8%9B%E0%B9%87%E0%B8%99%E0%B8%87%E0%B9%80%E0%B8%9B%E0%B9%87%E0%B8%99%E0%B8%82%E0%B9%89%E0%B8%AD%E0%B8%84%E0%B8%A7%E0%B8%B2%E0%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# เปลี่ยนเสียงเป็นข้อความง่ายๆ ด้วย Whisper: คู่มือการถอดเสียงและแปลภาษา
เปิดการใช้งาน gpu ของ colab ก่อนนะครับ 👉 https://www.geeksforgeeks.org/how-to-use-gpu-in-google-colab/


## การตั้งค่า
เราใช้ Python 3.9.9 และ [PyTorch](https://pytorch.org/) 1.10.1 เพื่อฝึกและทดสอบโมเดลของเรา

อย่างไรก็ตาม โค้ดเบสนี้ควรจะสามารถใช้งานได้กับ Python 3.8-3.11 และ PyTorch เวอร์ชันล่าสุด

โค้ดเบสนี้ยังพึ่งพาแพ็กเกจ Python บางตัว โดยเฉพาะ OpenAI's [tiktoken](https://github.com/openai/tiktoken) จาก OpenAI สำหรับการใช้งาน tokenizer ที่รวดเร็ว

คุณสามารถดาวน์โหลดและติดตั้ง (หรืออัปเดตเป็น) รุ่นล่าสุดของ Whisper ด้วยคำสั่งต่อไปนี้:

In [ ]:
pip install -U openai-whisper

### นอกจากนี้ยังจำเป็นต้องใช้เครื่องมือบรรทัดคำสั่ง [ffmpeg](https://ffmpeg.org/) ซึ่งสามารถติดตั้งได้จากตัวจัดการแพ็กเกจ:

In [ ]:
!apt update && apt install -y ffmpeg

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
45 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
f

### หากการติดตั้งล้มเหลวพร้อมข้อความ "No module named 'setuptools_rust'"

---

ให้ติดตั้ง setuptools_rust:

### คุณอาจต้องติดตั้ง rust ด้วย ในกรณีที่ tiktoken ไม่มีไบนารีสำเร็จรูปสำหรับระบบปฏิบัติการของคุณ

หากคุณพบข้อผิดพลาดในระหว่างการติดตั้งด้วยคำสั่ง `pip install` ข้างต้น

โปรดดูขั้นตอนการติดตั้งใน Getting started page เพื่อติดตั้งสภาพแวดล้อมการพัฒนา Rust

นอกจากนี้ คุณอาจต้องตั้งค่าตัวแปรสภาพแวดล้อม `PATH` ด้วย เช่น:
export PATH="$HOME/.cargo/bin:$PATH"

หากการติดตั้งล้มเหลวด้วยข้อผิดพลาด `No module named 'setuptools_rust'`

คุณจำเป็นต้องติดตั้ง `setuptools_rust` เช่น ด้วยคำสั่ง:

In [ ]:
pip install setuptools-rust

## โมเดลและภาษาที่ใช้ได้

มีโมเดลขนาด 5 ขนาด, 4 ขนาดสำหรับภาษาอังกฤษเท่านั้น พร้อมการปรับสมดุลความเร็วและความแม่นยำ ดังนี้:

| ขนาด    | พารามิเตอร์ | โมเดลภาษาอังกฤษ | โมเดลหลากภาษา | VRAM ที่ต้องการ | ความเร็วเทียบกับโมเดลใหญ่ |
|---------|--------------|-------------------|---------------|-----------------|------------------------|
| tiny    | 39 M         | tiny.en           | tiny          | ~1 GB           | ~32x                   |
| base    | 74 M         | base.en           | base          | ~1 GB           | ~16x                   |
| small   | 244 M        | small.en          | small         | ~2 GB           | ~6x                    |
| medium  | 769 M        | medium.en         | medium        | ~5 GB           | ~2x                    |
| large   | 1550 M       | N/A               | large         | ~10 GB          | 1x                     |

"en" โมเดลสำหรับภาษาอังกฤษทำงานได้ดีกว่า โดยเฉพาะสำหรับ tiny.en และ base.en การต่างกันนี้น้อยลงสำหรับ small.en และ medium.en

Whisper แสดงประสิทธิภาพแตกต่างกันไปตามภาษา ดังที่เห็นจาก WERs (Word Error Rates) และ CER (Character Error Rates) ที่วัดจากชุดข้อมูล Common Voice 15 และ Fleurs สามารถดูข้อมูลเพิ่มเติมได้ในเอกสารฉบับเต็ม.


## การใช้งานใน Python
การถอดเสียงสามารถทำได้ใน Python:

### ใช้แบบง่ายๆ
ไฟล์ เสียง mp3 ให้โหลดของตัวเองเข้าไปนะครับ แล้วแก้ path ให้ตรงกับชื่อไฟล์นะครับ

In [ ]:
import whisper
import torch

model = whisper.load_model("large")
result = model.transcribe("sample audio.mp3")
print(result["text"])

# นำข้อมูลออกจาก gpu memory
torch.cuda.empty_cache()

สวัสดีครับ ทดสอบ ทดสอบ คลิปนี้จะใช้ทดสอบการใช้ Model Whisper ในการถอยความจากเสียงนี้ออกมาเป็นภาษาไทยแบบตัวหนังสือนะครับ


### ภายในการทำงานของเมธอด `transcribe()`
เมธอด `transcribe()` จะอ่านไฟล์เสียงทั้งหมดและประมวลผลด้วยการเลื่อนหน้าต่างขนาด 30 วินาที

โดยจะทำการคาดการณ์อัตโนมัติแบบต่อเนื่องในแต่ละหน้าต่าง

ตัวอย่างการใช้งาน `whisper.detect_language()` และ `whisper.decode()`

ซึ่งเป็นการเรียกใช้โมเดลในระดับล่างสุด มีดังนี้:

In [ ]:
import whisper
import torch

model = whisper.load_model("base") # โมเดลขนาดเล็กให้ผลไม่ค่อยดีเท่าไหร่ครับ

# โหลดไฟล์เสียงและปรับขนาดให้เหมาะสมกับ 30 วินาที
audio = whisper.load_audio("sample audio.mp3")
audio = whisper.pad_or_trim(audio)

# สร้าง log-Mel spectrogram และย้ายไปยังอุปกรณ์เดียวกับโมเดล
mel = whisper.log_mel_spectrogram(audio).to(model.device)

# ตรวจสอบภาษาที่ใช้พูด
_, probs = model.detect_language(mel)
print(f"Detected language: {max(probs, key=probs.get)}")

# ถอดรหัสเสียง
options = whisper.DecodingOptions()
result = whisper.decode(model, mel, options)

# แสดงข้อความที่จดจำได้
print(result.text)

# นำข้อมูลออกจาก gpu memory
torch.cuda.empty_cache()

Detected language: th
สวัสดีครับ ทุกสอบทุกสอบคิดนี้จะใช้ทุกสอบบ้านชายโมเตอร์วิสเบิร์นี้การท่อยพราบจากเสียงมีออกมาเป็นมาสาฮาถ้ายแบบตัวนั้นสื้อดครับ


## การดาวน์โหลดและถอดเสียงจาก YouTube ด้วย Whisper

### ขั้นตอนที่ 1: ติดตั้ง yt-dlp สำหรับการดาวน์โหลดเสียงจาก YouTube

In [ ]:
!pip install yt-dlp

### ขั้นตอนที่ 2: ฟังก์ชั่นสำหรับการดาวน์โหลดเสียงจากวิดีโอ YouTube

In [ ]:
def download_audio(youtube_url, output_path="audio.mp3"):
    # คำสั่งเพื่อดึงเสียงเป็นไฟล์ MP3
    command = f"yt-dlp -x --audio-format mp3 -o '{output_path}' {youtube_url}"
    # รันคำสั่งนี้
    !{command}

In [ ]:
import os
import subprocess

def download_audio(youtube_url, output_path="audio.mp3"):
    # Check if file exists and delete it
    if os.path.exists(output_path):
        os.remove(output_path)
        print(f"Deleted existing {output_path}")

    # Command to download audio as an MP3 file
    command = f"yt-dlp -x --audio-format mp3 -o '{output_path}' {youtube_url}"

    # Run the command to download the file
    subprocess.run(command, shell=True)
    print(f"Downloaded audio to {output_path}")

In [ ]:
import os
import subprocess

def download_audio(youtube_url, output_path="audio.mp3"):
    # Define alternative file paths that might exist
    alternative_audio_path = "audio.webm"

    # Check if file exists and delete it
    if os.path.exists(output_path):
        os.remove(output_path)
        print(f"Deleted existing {output_path}")

    if os.path.exists(alternative_audio_path):
        os.remove(alternative_audio_path)
        print(f"Deleted existing {alternative_audio_path}")

    # Command to download audio as an MP3 file
    command = f"yt-dlp -x --audio-format mp3 -o '{output_path}' {youtube_url}"

    # Run the command to download the file
    subprocess.run(command, shell=True)
    print(f"Downloaded audio to {output_path}")

### ดาวน์โหลดเสียงจากวิดีโอ

In [ ]:
# URL ของวิดีโอ YouTube
youtube_url = "https://www.youtube.com/shorts/RoAA6_JMPqc"  # แทนที่ด้วย URL ที่ต้องการ
download_audio(youtube_url)


[youtube] Extracting URL: https://www.youtube.com/shorts/RoAA6_JMPqc
[youtube] RoAA6_JMPqc: Downloading webpage
[youtube] RoAA6_JMPqc: Downloading ios player API JSON
[youtube] RoAA6_JMPqc: Downloading android player API JSON
[youtube] RoAA6_JMPqc: Downloading m3u8 information
[info] RoAA6_JMPqc: Downloading 1 format(s): 251
[download] Destination: audio.webm
[download] 100% of 1006.48KiB in 00:00:00 at 4.08MiB/s
[ExtractAudio] Destination: audio.mp3
Deleting original file audio.webm (pass -k to keep)


### ขั้นตอนที่ 3: ถอดเสียงจากไฟล์ที่ดาวน์โหลดโดยใช้ Whisper
ขนาดเล็กให้ผลสำหรับภาษาไทยไม่ค่อยดีเท่าไหร่ครับ

#### model ขนาด tiny ให้ผลไม่ค่อยดีเท่าไหร่สำหรับภาษาไทยครับ

In [ ]:
import whisper
import torch

# โหลดโมเดล Whisper
model = whisper.load_model("tiny") # หรือขนาดอื่นๆ ที่ต้องการ ['tiny', 'base', 'small', 'medium', 'large']

# ถอดเสียงจากไฟล์เสียง
result = model.transcribe("audio.mp3")

# แสดงผลการถอดเสียง
print(result["text"])

# นำข้อมูลออกจาก gpu memory
torch.cuda.empty_cache()


ว่า รีชหน่อย aggressive that like thatWatching instead of you偷ลี Coverไม่มีน Both sidesคนนี่ลูกดินปว่าส Bellaเหรอ กับมีคomm specificsบ่าก็รีน นี้ เดี๋ยวแไหมร่ พวกใจถ้าสงพพมาปบistical ปตั 온ความาพลาของพัว asking you それ วิวันนี้กันอยู่ถ้าบคกี้ว่าสับเสร็ impeachmentถ้า I'll Sharpปล่าง4 4 Plus F10 ด้วยเราเมื่อเพราะ koy้าจะมันอะไร filling Burning and Painting is something we must say that someone knows each other's feelings.มา cuadιαสร้างตroseไหนหน้าไม่ทำย it's your father and well.เรารักหน้าจะเป็นมีากมากปามที่ Than℃ company of good lights and servants say it if they get mugged around their mind.Brianฝson and the gewordenall whom he's taking care of was when done by a maker of the vehicle that caught a closeลูกคนที่กินมาก ล้องปลาจายดู


#### model ขนาด base ก็ยังไงให้ผลไม่ค่อยดีสำหรับภาษาไทยครับ

In [ ]:
import whisper
import torch

# โหลดโมเดล Whisper
model = whisper.load_model("base") # หรือขนาดอื่นๆ ที่ต้องการ ['tiny', 'base', 'small', 'medium', 'large']
# ถอดเสียงจากไฟล์เสียง
result = model.transcribe("audio.mp3")

# แสดงผลการถอดเสียง
print(result["text"])


# นำข้อมูลออกจาก gpu memory
torch.cuda.empty_cache()

วันนี้ผมจะเป็น...ผมอยากจะเป็นปลิสนและกลากที่นำ AI มาชาย usedK จริงๆนะB-2P นี้ที่ทำให้ทำคุณทางโลกต้องมาจ้างคุณไทยทำอย่างไรผมอยากรู้เลยบอกพูดทิ่งนี้ อยากรู้เลยผมทำเป็น feedback ทั้งสิ้งแพล ปลาป ใช่ไหมครับสิ่ง เป็นหาหลากของมา เป็นหาหลากของผล ปลาป ผมผมคือว่า ที่ผมแปลที่พระสานคิดสุดท้าคุณท้ากะคุณท้าคุณท้าคุณท้าคุณท้าคุณท้าคุณท้าคุณท้าคุณท้าคุณท้าคุณท้าคุณท้าคุณท้าคุณท้าคุณท้าคุณท้าคุณท้าคุณท้าคุณท้าคุณท้าคุณท้าคุณท้าคุณท้าคุณท้าคุณท้าคุณท้าคุณท้าคุณท้าคุณท้าคุณท้าคุณท้าคุณท้าคุณท้าคุณท้าคุณท้าคุณท้าคุณท้าคุณท้าคบอกว่าลูก คนท้ายกิ่งมาก ลองเปล่าใจดู


#### model ขนาด small ก็ยังไงให้ผลไม่ค่อยดีสำหรับภาษาไทยครับ

In [ ]:
import whisper
import torch

# โหลดโมเดล Whisper
model = whisper.load_model("small") # หรือขนาดอื่นๆ ที่ต้องการ ['tiny', 'base', 'small', 'medium', 'large']

# ถอดเสียงจากไฟล์เสียง
result = model.transcribe("audio.mp3")

# แสดงผลการถอดเสียง
print(result["text"])

# นำข้อมูลออกจาก gpu memory
torch.cuda.empty_cache()

วันนี้ผมจะเป็น...ผมไม่อยากจะเป็นประสัตว์เล็กๆ ที่นำ AI มาใช้Use Case จริงๆนะปี 2 ปีนี้ที่ทำให้ทั้งโลกต้องมาจ้างคนไทยทำอย่างไร?ผมอยากรู้เลยพอพูดถึงนี้อยากรู้เลยผมทำเป็นฟริลังซ์ที่แพร์ฟอร์มปับใช่มั้ยครับสิ่งปัญหาลักของภาษาของผมคือว่าที่ผมแพร์ที่ยังมีภาษาของกิจ สุดท้าคนทัก คุณทักเป็นภาษาของบ้านเราอยู่อีกภาษาภาษาฟรังเสธภาษาศเบนภาษาษาของกิจแต่คนไทยฟิเล็งวัน 102 มือแล้วที่ผมมีอยู่ในมือคู่ศาคกิจของทุกคนไหม? ก็ไม่ใช่สุดท้าว่าวันนี้เรามีที่นรู้อย่างไง?ที่ผมสาวันนำมาใช้ได้แล้วถ้าวันนี้ผมสาวัสแปลประสาศเบนภาษาศเบนภาษาศเบนภาษาศเบนภาษาศเบนภาษาศเบนภาษาศเบนภาษาศเบนภาษาศเบนภาษาศเบนภาษาศเบนภาษาศเบนภาษาศเบนภาษาศเบนภาษาศเบนภาษาษาศเบนภาษาษาศเบนภาษาษาศ


#### model ขนาด medium ให้ผลเริ่มจะพอใช้ได้กว่าขนาดเล็กครับ


In [ ]:
import whisper
import torch

# โหลดโมเดล Whisper
model = whisper.load_model("medium") # หรือขนาดอื่นๆ ที่ต้องการ ['tiny', 'base', 'small', 'medium', 'large']

# ถอดเสียงจากไฟล์เสียง
result = model.transcribe("audio.mp3")

# แสดงผลการถอดเสียง
print(result["text"])

# นำข้อมูลออกจาก gpu memory
torch.cuda.empty_cache()

วันนี้ผมจะเป็น ผมอยากจะเป็นบริษัทแรกแรกที่นำ AI มาใช้ use case จริงๆนะปี 2 ปีนี้ที่ทำให้ทั้งโลกต้องมาจ้างคนไทยทำยังไงผมอยากรู้เลยพอพูดถึงนี้อยากรู้เลยพอทำเป็น freelance ใน platform ปัปใช่ไหมครับสิ่งปัญหาลักของว่าปัญหาลักของ platform ของผมคือว่าถึงผมแปลทุกอย่างเป็นภาษาอังกฤษสุดท้ากลัวคนทักคนทักเป็นภาษาบ้านเราอยู่ภาษา ภาษาพระรังเสตร ภาษาสเปน ภาษาอังกฤษแต่คนไทย freelanceมัน 1 สาส 2 มุลลาดที่ผมมีอยู่ในมือผู้ภาษากฤษของทุกคนไหมก็ไม่ใช่แต่ว่าวันนี้เรามีทิกโรง AIที่ผมสามารถนำมาใช้ได้แล้วถ้าวันนี้ผมสามารถแปลภาษาตรงๆจริงๆ technical briefสามารถแปลภาษาตรงๆ เป็นภาษาไทย ภาษาอังกฤษ ภาษาฟรังเสตร ภาษาอะไรก็ได้ผมกำลังทำการธรราชน์ข้างใหญ่ออกนอกประเทศไทยบอกว่าโลกคนไทยเก่งมากลองเปิดใจดู


#### model ขนาด large ให้ผลดีใช้ได้ครับ แต่ช้ามากๆเมื่อรันด้วย CPU

In [ ]:
import whisper
import torch

# โหลดโมเดล Whisper
model = whisper.load_model("large") # หรือขนาดอื่นๆ ที่ต้องการ ['tiny', 'base', 'small', 'medium', 'large']

# ถอดเสียงจากไฟล์เสียง
result = model.transcribe("audio.mp3")

# แสดงผลการถอดเสียง
print(result["text"])

# นำข้อมูลออกจาก gpu memory
torch.cuda.empty_cache()

วันนี้ผมอยากจะเป็นบริเศษแรกๆ ที่นำ AI มาใช้Use Case จริงๆ นะ ปี 2 ปีนี้ที่ทำให้ทั้งคนทั้งโลกต้องมาจ้างคนไทยทำยังไง ผมอยากรู้เลย พอพูดถึงนี้อยากรู้เลยเพราะทำเป็น Freelancing Platform ปั๊บใช่ไหมครับสิ่งปัญหาหลักของแผนฟอร์มของผมคือว่าถึงผมแปลทุกอย่างมีภาษาอังกฤษ สุดท้ายคนทักคุณก็ทักเป็นภาษาของบ้านเราก็อยู่อยู่ภาษาฝรั่งเศส ภาษาสเปน ภาษาอังกฤษแต่คนไทย Freelance มัน 120,000 บาท ที่ผมมีอยู่ในมือผู้ศักดิ์อังกฤษค่องทุกคนไหม ก็ไม่ใช่แต่ว่าวันนี้เรามีเทคโนโลยี AIที่ผมสามารถนำมาใช้ได้และถ้าวันนี้ผมสามารถแปลภาษาแบบตรงๆจริงๆ Technical Briefสามารถแปลตรงๆๆเป็นภาษาไทย ภาษาอังกฤษ ฝรั่งเศส ภาษาอะไรก็ได้ผมกำลังนำ ผมสามารถทำการตลาดขั้งใหญ่ออกนอกประเทศไทยบอกว่า โลกคนไทยเก่งมาก ลองเปิดใจดู


### ทำเป็นฟังชั่นให้ใช้ง่ายขึ้น

In [ ]:
import whisper
import torch

def transcribe_audio(audio_file_path, model_size="large"):
    """
    ใช้โมเดล Whisper ในการถอดเสียงจากไฟล์เสียง

    Args:
        audio_file_path (str): เส้นทางไฟล์เสียงที่ต้องการถอดเสียง
        model_size (str, optional): ขนาดของโมเดล Whisper ที่จะโหลด
            สามารถเลือกได้จาก ['tiny', 'base', 'small', 'medium', 'large']
            ค่าเริ่มต้นคือ 'large'

    Returns:
        str: ข้อความที่ถูกถอดเสียงจากไฟล์เสียง
    """
    # ขจัดหน่วยความจำ GPU
    torch.cuda.empty_cache()

    # โหลดโมเดล Whisper
    model = whisper.load_model(model_size)

    # ถอดเสียงจากไฟล์
    result = model.transcribe(audio_file_path)

    # นำข้อความที่ถูกถอดเสียงออกมา
    transcribed_text = result["text"]

    # ขจัดหน่วยความจำ GPU
    torch.cuda.empty_cache()

    return transcribed_text

In [ ]:
# การใช้งานตัวอย่าง:
audio_file_path = "audio.mp3"
transcribed_text = transcribe_audio(audio_file_path)
print(transcribed_text)



```
# This is formatted as code
```

### ใช้งานจริง:
1. ใส่ url ของ youtube ที่ต้องการถอดข้อความ
2. เลือกขนาด model (มีผลกับความเร็วและความถูกต้อง)
3. กดปุ่ม "run" ทางด้านซ้าย (หรือกดปุ่ม Ctrl + Enter)

In [ ]:
youtube_url = "https://www.youtube.com/watch?v=IDqC9tm3bAs&t=2634s" # @param {type:"string"}
model_size =  "large" # @param ["tiny", "base", "small", "medium", "large"] {type:"string"}
audio_file_path = "audio.mp3"

download_audio(youtube_url)
transcribed_text = transcribe_audio(audio_file_path, model_size=model_size)
print(transcribed_text)

Deleted existing audio.mp3
Downloaded audio to audio.mp3
วันนี้ยินดีต้อนรับนะครับสกูรีโอเปิดบ้านนะครับกับ AI Club ของเรานะครับก็ช่วงนี้คนพูดกันเยอะเรื่องของ AI นะครับAi Club จริงมีหลายอย่างที่เราพยายามทำเรามีรายการ AI Sarth อยู่บน YouTube พึ่งไพลอทตอนแรกไปสอนเปลี่ยนหน้าโมเดลทำนู้นทำนี่ แล้วก็จะมีความรู้อื่นๆ ที่มาให้แต่วันนี้เรามาเป็นเวอร์ชันออนกราวด์มาเจอหน้ากันจริงๆ เราจัดไปแล้วก่อนหน้านี้หนึ่งครั้งเวอร์ชันที่แล้วเป็น AI สำหรับ Developerวันนี้เรามาโฟกัสในเรื่องของ Data กันนะครับวันนี้เราได้รับเกียรติจาก Data Expertลองมาคุยกันว่าในโลกของ Data เราสามารถเอา AI มาช่วยทำงานเรายังไงได้บ้างก็แนะนำอันดับแรกผม ต้า น่าจะรู้จักหลายท่านอยู่แล้วมองไปแล้วเคิ้นเขินรู้จักเกือบทุกคนเลยชื่อต้า ก็เป็น MD อยู่ที่สกูรีโอวันนี้ครึ่งตัวเป็น Data Expert แต่อีกครึ่งตัวเป็นพิธีกรแล้วกันน้องน้องบอกว่า Topic นี้ยาก พี่ต้า Modulate เองเลยละกันเดี๋ยวเป็นพิธีกรให้ด้วยครึ่งหนึ่งทางที่สอง พี่สีระ เอกบุตรน่าจะไม่เคยมีใครเห็นหน้าเขาเท่าไหร่ เขาไม่ค่อยออกหน้าแต่ว่าทุกคนรู้จักเขาในนามเทป Excelใครเคยดูวีดีโอ ดูคลิปสอน ดู

In [1]:
long = 'https://www.youtube.com/watch?v=IDqC9tm3bAs&t=2634s'
short = 'https://www.youtube.com/shorts/jM46moNlUaM'
short2 = 'https://www.youtube.com/shorts/PEcnPHzsceg'